In [70]:
import numpy as np
import pandas as pd

from collections import Counter
from scipy.sparse import lil_matrix
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score

In [71]:
!pip install tokenizers
from tokenizers import CharBPETokenizer

Номер 1

In [72]:
with open('lenta.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [73]:
text = text[:100000]
#text

In [74]:
def freq(text):
    frequencies = {}
    for i in range(len(text)-1):
        current_symbol = text[i]
        next_symbol = text[i+1]
        pair = current_symbol + next_symbol
        if pair in frequencies:
            frequencies[pair] += 1
        else:
            frequencies[pair] = 1
    return frequencies

In [75]:
def sort_dict(dictionary, k):
    new_dict = {}
    keys = list(dictionary.keys())
    keys = sorted(keys, key=lambda x: dictionary[x], reverse=True)
    keys = keys[:k]
    for key in keys:
        new_dict[key] = dictionary[key]
    return new_dict

In [76]:
def update_text(text, dictionary):
    new_text = ()
    flag = True
    for i in range(len(text)):
        if flag:
            current_symbol = text[i]
            if i < len(text)-1:
                next_symbol = text[i+1]
            else:
                new_text += tuple(current_symbol, )
                break
            pair = current_symbol + next_symbol
            if pair in dictionary: 
                new_text += (pair, )
                flag = False
            else:
                new_text += (current_symbol, )
        else:
            flag = True
    return new_text

In [77]:
def bpe(text, n, k):
    new_text = text
    for i in range(n):
        frequencies = freq(new_text)
        top_freq = sort_dict(frequencies, k)
        new_text = update_text(new_text, top_freq)
    return new_text

Попробуем разные параметры n и k и выведем для них количество токенов и самый длинный токен

In [78]:
def show(result):
    print('/'.join(result[:57]))
    print('самые частотные токены', Counter(result).most_common(10))
    print('самый длинный токен', sorted(result, key=lambda x: len(x), reverse=True)[0])
    print('всего токенов', len(result))

In [79]:
for n in range(6, 9, 2):
    for k in [100, 200, 450]:
        result = bpe(text, n, k)
        print('n = ' + str(n), 'k = ' + str(k))
        show(result)

n = 6 k = 100
Б/о/и /у /С/о/по/ц/ки/на/ и/ Д/ру/ск/ени/к/ за/кон/ч/ил/ис/ь /от/ступ/лени/ем/ г/ер/м/ан/це/в/. Н/е/пр/ия/тель/, пр/и/бли/зи/вш/ис/ь /с/ с/е/ве/ра/ к/ О/со/в/ц/у/ на/ч
самые частотные токены [(' ', 525), ('с', 456), ('и', 448), (', ', 418), ('у', 412), ('б', 396), ('на', 386), ('к', 374), ('и ', 368), ('не', 366)]
самый длинный токен  сегодня 
всего токенов 45752
n = 6 k = 200
Б/о/и /у /С/оп/о/ц/ки/на/ и/ Д/ру/ск/ен/ик/ зак/он/чи/лись /от/ступле/нием/ г/ер/ма/н/це/в. /Н/е/прия/тель/, пр/ибл/из/ив/ши/сь /с /се/ве/ра/ к/ О/со/в/ц/у /нача/л /ар/ти/ллери/йс/ку/ю 
самые частотные токены [('  ', 291), ('на', 287), (', ', 286), (' в', 281), (' с', 256), ('не', 248), ('и ', 241), ('е ', 226), ('"', 226), ('по', 218)]
самый длинный токен  представител
всего токенов 39476
n = 6 k = 450
Бои/ у/ С/оп/оц/ки/на/ и/ Д/ру/ск/ен/ик/ закон/чи/лись/ отс/туплен/ием /ге/рм/ан/цев./ Н/епри/ятел/ь/, пр/иб/лизи/вш/ись /с /се/ве/ра/ к/ О/со/в/цу /нача/л /артиллер/ийск/ую/ бор/ь/бу/ с/ к/ре/пост/ь

Номер 2

In [80]:
data = pd.read_csv('dataset_ok.csv')

In [81]:
data

,text,label
0,"наебалово века, для долбаёбов\n",INSULT
1,вся дума в таком же положении😁\n,NORMAL
2,а в каком месте массовое столкновение? шрайбик...,NORMAL
3,"значит ли это, что контроль за вывозом крупног...",NORMAL
4,вам не нужен щеночек? очень хорошие 🐶🥰\n,NORMAL
...,...,...
71982,"царствие небесное ,🙏, одноклассник,\n",NORMAL
71983,здоровье вам маленькие детки\n,NORMAL
71984,я тоже дочери покупала эффекта ноль это развод...,NORMAL
71985,почему он держит чубайса до сих пор?\n,NORMAL


In [82]:
data['text'].to_csv('corpus.txt', index=None)

In [83]:
tok_sub = CharBPETokenizer()
tok_sub.train('corpus.txt', vocab_size=2000, min_frequency=10,)

In [84]:
data['tokenized'] = data['text'].apply(lambda x: tok_sub.encode(x).tokens)
data

,text,label,tokenized
0,"наебалово века, для долбаёбов\n",INSULT,"[на, еба, ло, во</w>, ве, ка</w>, ,</w>, для</..."
1,вся дума в таком же положении😁\n,NORMAL,"[вс, я</w>, ду, ма</w>, в</w>, та, ком</w>, же..."
2,а в каком месте массовое столкновение? шрайбик...,NORMAL,"[а</w>, в</w>, ка, ком</w>, ме, сте</w>, ма, с..."
3,"значит ли это, что контроль за вывозом крупног...",NORMAL,"[зна, чи, т</w>, ли</w>, это</w>, ,</w>, что</..."
4,вам не нужен щеночек? очень хорошие 🐶🥰\n,NORMAL,"[вам</w>, не</w>, ну, же, н</w>, щ, ен, о, че,..."
...,...,...,...
71982,"царствие небесное ,🙏, одноклассник,\n",NORMAL,"[ц, ар, стви, е</w>, не, бе, с, ное</w>, ,</w>..."
71983,здоровье вам маленькие детки\n,NORMAL,"[здоров, ье</w>, вам</w>, ма, лен, ь, кие</w>,..."
71984,я тоже дочери покупала эффекта ноль это развод...,NORMAL,"[я</w>, тоже</w>, до, че, ри</w>, по, ку, па, ..."
71985,почему он держит чубайса до сих пор?\n,NORMAL,"[почему</w>, он</w>, дер, жи, т</w>, чу, ба, й..."


In [85]:
#вычислим для каждого слова количество документов, в котором оно встречается
frqncs = {}
for document in data['tokenized']:
  for word in document:
    if word in frqncs:
      frqncs[word] += 1
    else:
      frqncs[word] = 1

In [86]:
doc_len = data.shape[0]
doc_len

71987

In [87]:
#вычислим idf
idf_dict = {}
for word in frqncs:
  idf = np.log(doc_len/frqncs[word])
  idf_dict[word] = idf

In [88]:
X = lil_matrix((doc_len, len(idf_dict.keys())))

In [89]:
for i in range(len(data['tokenized'])):
  document = data['tokenized'][i]
  length = len(document)
  #вычисляем tf
  tf_dict = {}
  for word in document:
    if word in tf_dict:
      tf_dict[word] += 1/length
    else:
      tf_dict[word] = 1/length
  #записываем в матрицу (можно конечно было сделать и за один проход)
  indices = list(idf_dict.keys())
  for word in document:
    index = indices.index(word)
    X[i, index] = tf_dict[word] * idf_dict[word]

In [90]:
y= data['label']

In [91]:
classifier = SGDClassifier(loss="log", max_iter=150)

In [92]:
cross_val_score(classifier, X, y, scoring="f1_micro")

array([0.87595499, 0.87796916, 0.87761339, 0.8796277 , 0.87879419])